## 문제 6

**Kaggle 형** train_prob.csv로 문제 target을 예측하는 모델을 만들고, 

test_prob.csv에 대한 target 예측하여 다음과 같은 형식의 answer6.csv를 만들어라.

id, target

0, 6.9

5, 7.8

...


**평가지표**

$RMSE(Y, \hat{Y}) = \sqrt{\frac{1}{n}\sum^{n}_{i=1}(y_i-\hat{y_i})^2}$

**강사: 멀티캠퍼스 강선구(sunku0316.kang@multicampus.com, sun9sun9@gmail.com)**

In [1]:
# 실행 환경 확인

import pandas as pd
import numpy as np
import sklearn
import scipy
import statsmodels
import mlxtend
import sys
import xgboost as xgb

print(sys.version)
for i in [pd, np, sklearn, scipy, mlxtend, statsmodels, xgb]:
    print(i.__name__, i.__version__)

3.7.4 (default, Oct 17 2019, 06:10:02) 
[GCC 8.3.0]
pandas 0.25.1
numpy 1.18.5
sklearn 0.21.3
scipy 1.5.2
mlxtend 0.15.0.0
statsmodels 0.11.1
xgboost 0.80


In [2]:
df_train = pd.read_csv('train_prob.csv', index_col='id')
df_test = pd.read_csv('test_prob.csv', index_col='id')
df_ans = pd.read_csv('test_prob_ans.csv', index_col='id')

In [3]:
# 처리 과정에 필요하 내용들을 list 형태로 구성합니다.
repl_list = [
    ('cat3', {'B': 'C'}, [83634, 147361, 9005]),
    ('cat4', {'A': 'B', 'D': 'B'}, [239397, 603]),
    ('cat6', {'D': 'A', 'E': 'B', 'G': 'C', 'H': 'B', 'I': 'A'}, [234203, 5145, 652]),
    ('cat7', {'A': 'B', 'C': 'B', 'F': 'D', 'I': 'B'}, [4606, 19784, 214027, 1583]),
    ('cat8', {'B': 'G', 'F': 'E'}, [30338, 96743, 2953, 76085, 33881]),
    ('cat9', {'C': 'H', 'D': 'B', 'E': 'L'}, [10678, 2846, 85944, 8320, 19987, 40070, 5501, 16743, 33793, 7819, 3331, 4968])
]
# 반복문 처리 내용들을 수행합니다.
for v, d, cnts in repl_list:
    print(v, d, cnts)
    # 치환후 내용을 s_repl에 저장합니다
    s_repl = df_train[v].replace(d)
    # 치환결과를 확인합니다.
    if (s_repl.nunique() != len(cnts)) or ((s_repl.value_counts().sort_index() != cnts).any()):
        print(s_repl.value_counts())
        break
    df_train[v] = s_repl
    if v != 'cat9': # test에 cat9가 train에 등장하지 않는 케이스를 만듭니다.
        df_test[v] = df_test[v].replace(d)
    
cat_cols = ['cat{}'.format(i) for i in range(10)]
cont_cols = ['cont{}'.format(i) for i in range(14)]

cat3 {'B': 'C'} [83634, 147361, 9005]
cat4 {'A': 'B', 'D': 'B'} [239397, 603]
cat6 {'D': 'A', 'E': 'B', 'G': 'C', 'H': 'B', 'I': 'A'} [234203, 5145, 652]
cat7 {'A': 'B', 'C': 'B', 'F': 'D', 'I': 'B'} [4606, 19784, 214027, 1583]
cat8 {'B': 'G', 'F': 'E'} [30338, 96743, 2953, 76085, 33881]
cat9 {'C': 'H', 'D': 'B', 'E': 'L'} [10678, 2846, 85944, 8320, 19987, 40070, 5501, 16743, 33793, 7819, 3331, 4968]


In [4]:
from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from scipy.stats import norm

import os

X_all = df_test.columns.tolist()
df_train['targetA'] = df_train['target'] <= 7.45

if not os.path.isfile('targetA_train.csv'):
    
    df_train_clf = df_train.assign(
        prob_A = 1 - norm.cdf(df_train['target'], loc=6.769, scale=0.616),
        prob_B = norm.cdf(df_train['target'], loc=8.123, scale=0.527)
    ).query('prob_B < 0.01 or prob_A < 0.01').copy()
    
    clf_xgb = make_pipeline( 
        ColumnTransformer([
            ('ohe', OneHotEncoder(handle_unknown='ignore'), cat_cols),
            ('pt', 'passthrough', cont_cols)
        ]),
        xgb.XGBClassifier(
            max_depth = 2, # 트리의 최대 깊이 2
            reg_alpha = 0.1, # L1 규제 0.1
            reg_lambda = 0.1, # L2 규제 0.1
            colsample_bytree=0.25, # 트리 당 컬럼 샘플링 비율 0.25
            n_estimators=500, # 트리의 수 500
            random_state=123, # random_state 123
        )
    )
    
    clf_xgb.fit(df_train_clf[X_all], df_train_clf['targetA'])
    df_targetA_train = pd.DataFrame({'targetA_prob': clf_xgb.predict_proba(df_train[X_all])[:, 1]}, index=df_train.index)
    
    targetA_prob_cv = cross_val_predict(clf_xgb, df_train_clf[X_all], df_train_clf['targetA'], cv=5, method='predict_proba')
    df_targetA_train.loc[df_targetA_train.index.isin(df_train_clf.index), 'targetA_prob'] = targetA_prob_cv[:, 1]
    
    df_targetA_train.to_csv('targetA_train.csv')
    df_targetA_test = pd.DataFrame({'targetA_prob': clf_xgb.predict_proba(df_test[X_all])[:, 1]}, index=df_test.index)
    df_targetA_test.to_csv('targetA_test.csv')
else:
    df_targetA_train = pd.read_csv('targetA_train.csv', index_col='id')
    df_targetA_test = pd.read_csv('targetA_test.csv', index_col='id')

In [5]:
df_train['targetA_prob'] = df_targetA_train['targetA_prob']
df_test['targetA_prob'] = df_targetA_test['targetA_prob']

In [6]:
q = [i for i in np.arange(0, 1.01, 0.01)]
for i in cont_cols:
    q_val = df_train[i].quantile(q)
    q_val.iloc[[0, -1]] = [-np.inf, np.inf]
    q_cut = pd.cut(df_train[i], bins=q_val)
    q_mean = df_train.groupby(q_cut)['target'].mean()
    df_train[i + '_q'] = q_cut.map(q_mean).astype('float')
    df_test[i + '_q'] = pd.cut(df_test[i], bins=q_val).map(q_mean).astype('float')

In [7]:
# 공통
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.metrics import make_scorer
from sklearn.model_selection import ShuffleSplit, KFold

s_hist = []
cv = KFold(n_splits=5, random_state=123)
ss = ShuffleSplit(n_splits=1, train_size=0.8, random_state=123)

df_ans = pd.read_csv('test_prob_ans.csv', index_col='id')
X_all = df_test.columns.tolist()  + ['targetA_prob']

cat_cols = ['cat{}'.format(i) for i in range(10)]
cont_cols = ['cont{}'.format(i) for i in range(14)]
cont_q_cols = ['cont{}_q'.format(i) for i in range(14)]

q = [i for i in np.arange(0, 1.01, 0.01)]
def eval_model(model_name, model, sp):
    score_train, score_valid = list(), list()
    for train_idx, valid_idx in sp.split(df_train):
        df_cv_train, df_valid = df_train.iloc[train_idx].copy(), df_train.iloc[valid_idx].copy()
        for i in cont_cols:
            q_val =  df_cv_train[i].quantile(q)
            q_val.iloc[[0, -1]] = [-np.inf, np.inf]
            q_cut = pd.cut(df_cv_train[i], bins=q_val)
            q_mean = df_cv_train.groupby(q_cut)['target'].mean()
            df_cv_train[i + '_q'] = q_cut.map(q_mean).astype('float')
            df_valid[i + '_q'] = pd.cut(df_valid[i], bins=q_val).map(q_mean).astype('float')
        model.fit(df_cv_train[X_all], df_cv_train['target'])
        score_valid.append((mean_squared_error(df_valid['target'], model.predict(df_valid[X_all]))) ** 0.5)
        score_train.append((mean_squared_error(df_cv_train['target'], model.predict(df_cv_train[X_all]))) ** 0.5)
        
    output = 'Valid: {:.5f}±{:.5f},  V.Train: {:.5f}±{:.5f}'.format( 
            np.mean(score_valid), np.std(score_valid),
            np.mean(score_train), np.std(score_train),
        )
    print(output)
    s_hist.append(pd.Series([model_name, output, np.mean(score_valid)], index=['model name', 'result', 'score']))
    df_result = pd.DataFrame(s_hist)
    display(df_result.groupby('model name').last())
    display(df_result.loc[df_result['score'].idxmin()])

def select_model(model):
    model.fit(df_train[X_all], df_train['target'])
    prd = model.predict(df_test[X_all])
    pd.DataFrame({
        'id': df_test.index.values,
        'target': prd
    }).to_csv('answer6.csv', index = None)
    return prd 

In [12]:
from sklearn.linear_model import LinearRegression

ct = ColumnTransformer([
    ('ohe', OneHotEncoder(handle_unknown='ignore'), cat_cols), # drop과 handle_unknown = 'ignore'를 동시에 쓸수 없어, drop을 포기합니다.
    ('std', StandardScaler(), cont_cols)
])

reg_lr = make_pipeline(ct, LinearRegression())

eval_model('baseline', reg_lr, cv)

Valid: 0.86325±0.00296,  V.Train: 0.86303±0.00074


,result,score
model name,,
baseline,"Valid: 0.86325±0.00296, V.Train: 0.86303±0.00074",0.863247


model name                                             baseline
result        Valid: 0.86325±0.00295,  V.Train: 0.86300±0.00073
score                                                  0.863246
Name: 0, dtype: object

In [13]:
prd = select_model(reg_lr)
print("baseline 채점 결과:",  mean_squared_error(df_ans['target'], prd) ** 0.5)

baseline 채점 결과: 534085184.9390879


In [14]:
# 모델의 상태를 봅니다.
# 완전한 다중공선성에 의해 굉장히 불안정한 회귀계수가 만들어집니다.
# 이로 인해 등장하지 않은 케이스에서 이상한 예측 값이 만들어저 굉장히 높은 RMSE를 얻게 된 것입니다.
ct = ColumnTransformer([
    ('ohe', OneHotEncoder(handle_unknown = 'ignore'), cat_cols),
    ('std', StandardScaler(), cont_cols)
])

reg_lr = make_pipeline(ct, LinearRegression())
reg_lr.fit(df_train[X_all], df_train['target'])
reg_lr[1].coef_

array([-5.79874229e+09, -5.79874229e+09,  3.19972035e+08,  3.19972035e+08,
       -5.44993806e+09, -5.44993806e+09,  5.35890416e+09,  5.35890416e+09,
        5.35890416e+09,  3.66690782e+11,  3.66690782e+11, -5.81229216e+10,
       -5.81229216e+10, -5.81229216e+10, -5.81229216e+10,  5.72112918e+09,
        5.72112918e+09,  5.72112918e+09,  1.56031518e+10,  1.56031518e+10,
        1.56031518e+10,  1.56031518e+10,  1.77241212e+10,  1.77241212e+10,
        1.77241212e+10,  1.77241212e+10,  1.77241212e+10, -1.47188070e+10,
       -1.47188070e+10, -1.47188070e+10, -1.47188070e+10, -1.47188070e+10,
       -1.47188070e+10, -1.47188070e+10, -1.47188070e+10, -1.47188070e+10,
       -1.47188070e+10, -1.47188070e+10, -1.47188070e+10, -5.82580566e-02,
        3.44848633e-02,  2.14004517e-03, -5.58471680e-03, -5.43212891e-03,
       -5.46493530e-02,  2.36053467e-02,  1.55181885e-02,  6.80541992e-02,
        2.73017883e-02, -2.11868286e-02,  5.16967773e-02,  1.57165527e-02,
       -5.84411621e-03])

In [20]:
# 원인을 탐색합니다.

# 차집합 연산을 이용하여 등장하지 않은 범주값을 뽑아냅니다.
pd.concat([
    df_train[cat_cols].apply(set, axis = 0).rename('train'),
    df_test[cat_cols].apply(set, axis = 0).rename('test')
], axis=1).apply(
    lambda x: x['test'] - x['train'], axis =1
)

cat0           {}
cat1           {}
cat2           {}
cat3           {}
cat4           {}
cat5           {}
cat6           {}
cat7           {}
cat8           {}
cat9    {E, C, D}
dtype: object

In [ ]:
df_test['cat9'].mode()

In [22]:
# 조치를 합니다.
# 미등장 범주값을 최빈값으로 치환
m = df_train['cat9'].mode()[0]
df_test['cat9'] = df_test['cat9'].replace({'E': m, 'C': m, 'D': m})

In [23]:
# 해결 되었는지 확인합니다.
pd.concat([
    df_train[cat_cols].apply(set, axis = 0).rename('train'),
    df_test[cat_cols].apply(set, axis = 0).rename('test')
], axis=1).apply(
    lambda x: x['test'] - x['train'], axis =1
)

cat0    {}
cat1    {}
cat2    {}
cat3    {}
cat4    {}
cat5    {}
cat6    {}
cat7    {}
cat8    {}
cat9    {}
dtype: object

In [24]:
from sklearn.linear_model import LinearRegression

#  drop ='first'사용합니다.
ct = ColumnTransformer([
    ('ohe', OneHotEncoder(drop='first'), cat_cols),
    ('std', StandardScaler(), cont_cols)
])

reg_lr = make_pipeline(ct, LinearRegression())
eval_model('baseline', reg_lr, cv)

Valid: 0.86325±0.00295,  V.Train: 0.86300±0.00073


,result,score
model name,,
baseline,"Valid: 0.86325±0.00295, V.Train: 0.86300±0.00073",0.863246


model name                                             baseline
result        Valid: 0.86325±0.00295,  V.Train: 0.86300±0.00073
score                                                  0.863246
Name: 0, dtype: object

In [25]:
# 모델의 상태를 봅니다.
# 등장하지 않은 범주값은 없으니, 에러가 해결됩니다.
# 지나치게 회귀 계수가 커졌던 문제가 해결 됩니다.

reg_lr = make_pipeline(ct, LinearRegression())
reg_lr.fit(df_train[X_all], df_train['target'])
reg_lr[1].coef_

array([-0.14737833,  0.16983335,  0.24147158,  0.10693804,  0.03994362,
        0.15713566,  0.03681214, -0.17486971,  0.03074087,  0.21519861,
        0.36870383, -0.00597853, -0.05547379, -0.05870304, -0.00092302,
        0.05854559,  0.07181603, -0.10018803,  0.03508515,  0.13365193,
        0.1647826 ,  0.08022279, -0.00404722,  0.02865399,  0.09003857,
        0.21092791,  0.09276305,  0.09655687,  0.07017521, -0.05824285,
        0.03454985,  0.00214071, -0.00554697, -0.00543278, -0.05465513,
        0.02359775,  0.01552286,  0.06808225,  0.0272966 , -0.02116763,
        0.05174013,  0.01574269, -0.0058063 ])

In [26]:
prd = select_model(reg_lr)
print("baseline 채점 결과:",  mean_squared_error(df_ans['target'], prd) ** 0.5)

baseline 채점 결과: 0.8657268588416209


**선형회귀**, **로지스틱회귀** 등 모수적 모델에서는 완전한 다중공선성은 모델에 큰 불안정을 야기시킬 수 있습니다. 

따라서 drop='first'등 범주를 하나 제거하여 공선성을 없애는 작업은 반드시 해주어야 합니다.